In [ ]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense
from keras.models import Sequential
from sklearn.metrics import classification_report

# 選擇數據處理模式：'N-R' 或 'R'
mode = 'N-R'  # 可以選擇 'N-R' 或 'R'

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2015train-A.txt',
    './dataset/train/twitter-2016train-A.txt',
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # Non-Regional 模式，直接將推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # Regional 模式，將推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 載入 GloVe 詞嵌入
embeddings_index = {}
with open(r'C:\Users\jk121\Documents\Code\LargeData\glove.twitter.27B.25d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

### 建立單一 CNN 模型並加入 Flatten 層

In [ ]:
from keras.layers import Flatten

cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 2s 3ms/step - loss: 1.0291 - accuracy: 0.4510 - val_loss: 0.9808 - val_accuracy: 0.4751
Epoch 2/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9392 - accuracy: 0.5300 - val_loss: 0.9250 - val_accuracy: 0.5190
Epoch 3/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9031 - accuracy: 0.5535 - val_loss: 0.9144 - val_accuracy: 0.5277
Epoch 4/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8850 - accuracy: 0.5650 - val_loss: 0.9008 - val_accuracy: 0.5354
Epoch 5/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8741 - accuracy: 0.5724 - val_loss: 0.8977 - val_accuracy: 0.5400
Epoch 6/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8649 - accuracy: 0.5796 - val_loss: 0.8945 - val_accuracy: 0.5487
Epoch 7/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8574 - accuracy: 0.5808 - val_loss: 0.8917 - val_accuracy: 0.5558
Epoch 

### 單一 LSTM 模型 (Single LSTM)

In [ ]:
from keras.layers import LSTM

# 建立單一 LSTM 模型
lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 25s 56ms/step - loss: 0.9334 - accuracy: 0.5286 - val_loss: 0.8693 - val_accuracy: 0.5719
Epoch 2/10
405/405 [==============================] - 22s 55ms/step - loss: 0.8915 - accuracy: 0.5600 - val_loss: 0.8624 - val_accuracy: 0.5697
Epoch 3/10
405/405 [==============================] - 23s 57ms/step - loss: 0.8792 - accuracy: 0.5627 - val_loss: 0.8508 - val_accuracy: 0.5886
Epoch 4/10
405/405 [==============================] - 25s 62ms/step - loss: 0.8611 - accuracy: 0.5778 - val_loss: 0.8448 - val_accuracy: 0.5947
Epoch 5/10
405/405 [==============================] - 25s 61ms/step - loss: 0.8464 - accuracy: 0.5937 - val_loss: 0.8346 - val_accuracy: 0.6040
Epoch 6/10
405/405 [==============================] - 25s 61ms/step - loss: 0.8373 - accuracy: 0.5995 - val_loss: 0.8365 - val_accuracy: 0.6056
Epoch 7/10
405/405 [==============================] - 25s 63ms/step - loss: 0.8287 - accuracy: 0.6002 - val_loss: 0.8282 - val_accuracy:

### CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

In [ ]:
# 建立 CNN + LSTM 模型
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 13s 28ms/step - loss: 0.9690 - accuracy: 0.4961 - val_loss: 0.9151 - val_accuracy: 0.5335
Epoch 2/10
405/405 [==============================] - 11s 27ms/step - loss: 0.9100 - accuracy: 0.5431 - val_loss: 0.8898 - val_accuracy: 0.5546
Epoch 3/10
405/405 [==============================] - 10s 25ms/step - loss: 0.8890 - accuracy: 0.5595 - val_loss: 0.9054 - val_accuracy: 0.5403
Epoch 4/10
405/405 [==============================] - 10s 25ms/step - loss: 0.8768 - accuracy: 0.5706 - val_loss: 0.8710 - val_accuracy: 0.5638
Epoch 5/10
405/405 [==============================] - 9s 22ms/step - loss: 0.8672 - accuracy: 0.5792 - val_loss: 0.8703 - val_accuracy: 0.5743
Epoch 6/10
405/405 [==============================] - 9s 22ms/step - loss: 0.8619 - accuracy: 0.5774 - val_loss: 0.8653 - val_accuracy: 0.5697
Epoch 7/10
405/405 [==============================] - 9s 22ms/step - loss: 0.8514 - accuracy: 0.5875 - val_loss: 0.8611 - val_accuracy: 0.

### CNN + LSTM 組合模型 + SVM 組合模型

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

102/102 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0       0.55      0.66      0.60      1372
           1       1.00      0.00      0.00       466
           2       0.53      0.61      0.57      1397

    accuracy                           0.54      3235
   macro avg       0.70      0.42      0.39      3235
weighted avg       0.61      0.54      0.50      3235



### 單一 3 層 CNN 和 LSTM 網絡

In [ ]:
three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 7s 11ms/step - loss: 0.9564 - accuracy: 0.5063 - val_loss: 0.9022 - val_accuracy: 0.5410
Epoch 2/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8926 - accuracy: 0.5594 - val_loss: 0.9086 - val_accuracy: 0.5385
Epoch 3/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8647 - accuracy: 0.5781 - val_loss: 0.8895 - val_accuracy: 0.5635
Epoch 4/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8348 - accuracy: 0.6003 - val_loss: 0.8660 - val_accuracy: 0.5734
Epoch 5/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8082 - accuracy: 0.6170 - val_loss: 0.8944 - val_accuracy: 0.5657
Epoch 6/10
405/405 [==============================] - 4s 10ms/step - loss: 0.7746 - accuracy: 0.6377 - val_loss: 0.9161 - val_accuracy: 0.5589
Epoch 7/10
405/405 [==============================] - 4s 10ms/step - loss: 0.7345 - accuracy: 0.6576 - val_loss: 0.9285 - val_accuracy: 0.5706

### 多重 CNN 和 LSTM 網絡

In [ ]:
multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 10s 19ms/step - loss: 0.9467 - accuracy: 0.5209 - val_loss: 0.8923 - val_accuracy: 0.5450
Epoch 2/10
405/405 [==============================] - 7s 17ms/step - loss: 0.8877 - accuracy: 0.5626 - val_loss: 0.8714 - val_accuracy: 0.5648
Epoch 3/10
405/405 [==============================] - 7s 18ms/step - loss: 0.8576 - accuracy: 0.5871 - val_loss: 0.8493 - val_accuracy: 0.5833
Epoch 4/10
405/405 [==============================] - 7s 18ms/step - loss: 0.8298 - accuracy: 0.6033 - val_loss: 0.8459 - val_accuracy: 0.5774
Epoch 5/10
405/405 [==============================] - 7s 18ms/step - loss: 0.7958 - accuracy: 0.6254 - val_loss: 0.8477 - val_accuracy: 0.5879
Epoch 6/10
405/405 [==============================] - 7s 17ms/step - loss: 0.7666 - accuracy: 0.6452 - val_loss: 0.8480 - val_accuracy: 0.5978
Epoch 7/10
405/405 [==============================] - 7s 17ms/step - loss: 0.7361 - accuracy: 0.6655 - val_loss: 0.8500 - val_accuracy: 0.590

### 單一 3 層 CNN 和雙向 LSTM 網絡

In [ ]:
from keras.layers import Bidirectional

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 12s 19ms/step - loss: 0.9597 - accuracy: 0.5024 - val_loss: 0.9016 - val_accuracy: 0.5533
Epoch 2/10
405/405 [==============================] - 7s 18ms/step - loss: 0.8969 - accuracy: 0.5546 - val_loss: 0.8756 - val_accuracy: 0.5709
Epoch 3/10
405/405 [==============================] - 7s 18ms/step - loss: 0.8666 - accuracy: 0.5781 - val_loss: 0.8701 - val_accuracy: 0.5635
Epoch 4/10
405/405 [==============================] - 7s 18ms/step - loss: 0.8313 - accuracy: 0.5995 - val_loss: 0.8684 - val_accuracy: 0.5623
Epoch 5/10
405/405 [==============================] - 7s 18ms/step - loss: 0.8024 - accuracy: 0.6217 - val_loss: 0.8837 - val_accuracy: 0.5774
Epoch 6/10
405/405 [==============================] - 7s 18ms/step - loss: 0.7593 - accuracy: 0.6447 - val_loss: 0.9345 - val_accuracy: 0.5716
Epoch 7/10
405/405 [==============================] - 7s 18ms/step - loss: 0.7203 - accuracy: 0.6704 - val_loss: 0.8796 - val_accuracy: 0.576

### 多重 CNN 和雙向 LSTM 網絡

In [ ]:
multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 98s 232ms/step - loss: 0.9483 - accuracy: 0.5122 - val_loss: 0.9197 - val_accuracy: 0.5385
Epoch 2/10
405/405 [==============================] - 26s 64ms/step - loss: 0.8871 - accuracy: 0.5648 - val_loss: 0.8795 - val_accuracy: 0.5654
Epoch 3/10
405/405 [==============================] - 13s 32ms/step - loss: 0.8509 - accuracy: 0.5918 - val_loss: 0.8601 - val_accuracy: 0.5777
Epoch 4/10
405/405 [==============================] - 13s 32ms/step - loss: 0.8195 - accuracy: 0.6140 - val_loss: 0.8419 - val_accuracy: 0.5938
Epoch 5/10
405/405 [==============================] - 13s 33ms/step - loss: 0.7832 - accuracy: 0.6391 - val_loss: 0.8660 - val_accuracy: 0.5895
Epoch 6/10
405/405 [==============================] - 13s 32ms/step - loss: 0.7493 - accuracy: 0.6571 - val_loss: 0.8711 - val_accuracy: 0.5818
Epoch 7/10
405/405 [==============================] - 14s 35ms/step - loss: 0.7180 - accuracy: 0.6799 - val_loss: 0.8688 - val_accuracy